In [ ]:
import json
from pathlib import Path
from pypdf import PdfReader, PdfWriter

In [4]:
def get_value(data, path):
    """Récupère la valeur d’un chemin de type 'system.details.level.value'."""
    for part in path.split('.'):
        if '[' in part:
            name, idx = part[:-1].split('[')
            data = data[name][int(idx)]
        else:
            data = data[part]
    return data

# Exemple de correspondances PDF ↔ JSON
FIELD_MAP = {
    "NomPerso": "name",                                     # Nom du personnage
    "Classe": "system.details.class.value",                 # Classe actuelle
    "Niveau": "system.details.level.value",                 # Niveau
    "PointsDeVie": "system.attributes.hp.value",            # Points de vie
    # Ajouter ici toutes les autres correspondances…
}

In [5]:
pdf_path = Path("Feuille de personnage Remaster (éditable) 1.1.pdf")
json_path = Path("perso.json")

In [11]:
# Chargement du JSON
with open(json_path, encoding="utf-8") as f:
    character = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'perso.json'

In [ ]:
# Lecture du PDF
reader = PdfReader(str(pdf_path))
writer = PdfWriter()

# Copie des pages
for page in reader.pages:
    writer.add_page(page)

# Remplissage des champs
for pdf_field, json_path in FIELD_MAP.items():
    try:
        value = get_value(character, json_path)
    except (KeyError, IndexError, TypeError):
        continue  # Ignore les chemins manquants

    writer.update_page_form_field_values(
        writer.pages[0], {pdf_field: str(value)}
    )

# Sauvegarde du PDF rempli
with open("feuille_remplie.pdf", "wb") as f:
    writer.write(f)
